# Melanoma recognition using transfer learning

## Introduction

In this notebook, the main steps of a possible way to use transfer learning for melanoma recognition are summarized. The dataset used is available on Kaggle: https://www.kaggle.com/nroman/melanoma-external-malignant-256
This dataset contains almost 40,000 images (256 x 256 pixels) of melanoma and harmless skin marks.

The main objective of this notebook is to provide a way to build an algorithm to recognize melanoma on photographs. For this purpose, I choose to use transfer learning, which consists in using a pre-trained algorithm, adapting a bit its structure for the data and fine-tuning it using a training dataset. The main advantage of this method is that it enables to use an already proven algorithm, which has been trained for a very long time and which has already learned to recognize multiple image patterns in a very large dataset. 

In the following, the ResNext50 model is used (https://pytorch.org/hub/pytorch_vision_resnext/). This CNN is namely more efficient than numerous pre-trained alternatives available for transfer learning, and was shown to be among the most accurate ones on the famous ImageNet dataset.

Running this notebook on Kaggle allows to use a GPU, which is very useful to speed up the CNN work. We will have to indicate to PyTorch which objects should be transferred to the GPU (mainly the model and the image and target values tensors). Thus, each time the .cuda() function appears in the notebook, it means the object to which it is applied is transferred to the GPU.

## Data importation and processing

We first import the main libraries needed in the following. The neural network will be trained using the PyTorch library.

In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split

from PIL import Image

We introduce below the MelanomaDataset class, which is a subclass of the PyTorch Dataset class and a helper to build the datasets for the training, validation and evaluation steps. This class takes the following arguments:
- path: path to the folder containing the data
- mode: "train", "val" or "test", depending on the dataset we want to build
- frac_val: fraction of the whole dataset which should be kept for validation
- frac_test: fraction of the whole dataset which should be kept for evaluation
- random_state: seed to fix the train / val / test splitting of the dataset and to obtain reproducible results.

As any subclass of the Dataset class, this class must implement three methods: init, len and getitem. The init method only aims at initializing an instance of the class. The len method returns the number of instances in the dataset. Finally, the getitem method takes as argument an index and returns the corresponding element (both the image and the target value) of the dataset.
Note that the result given by these two methods depends on the mode initially selected when creating the class instance. If the value of mode is "train" (resp. "val", "test"), then the result of len is the number of instances in the training (resp. validation, evaluation) dataset. Similarly, the getitem method returns the element at position id in the training, validation or evaluation dataset.

Another method called train_val_test_split is implemented. This simply aims at splitting the dataset in three independent parts: one for training, one for validation, one for evaluation.

Note that the images are preprocessed; they are namely cropped to a square of size 224 x 224 and normalized using mean [0.485, 0.456, 0.406] and standard deviation [0.229, 0.224, 0.225]. This is indeed required by the network that will be used.

In [3]:
class MelanomaDataset(Dataset):
    def __init__(self, path, mode="train", frac_val=0.2, frac_test=0.2, random_state=1234):
        self.path = path
        self.mode = mode
        self.frac_val = frac_val
        self.frac_test = frac_test
        self.random_state = random_state
        labels = pd.read_csv(path+"/train_concat.csv",header=0,sep=",")
        image_name = labels["image_name"]
        target = labels["target"]
        self.image_name = image_name
        self.target = target
        path_images = path+"/train/train"
        self.path_images = path_images
        xtrain, ytrain, xval, yval, xtest, ytest = self.__train_val_test_split__()
        self.train = (xtrain, ytrain)
        self.val = (xval, yval)
        self.test = (xtest, ytest)
        self.preprocess = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def __len__(self):
        if self.mode=="train":
            return len(self.train[0])
        if self.mode=="val":
            return len(self.val[0])
        if self.mode=="test":
            return len(self.test[0])
    def __getitem__(self, id):
        if self.mode=="train":
            image_id = self.train[0].iloc[id]
            file = self.path_images+"/"+image_id+".jpg"
            x = self.preprocess(Image.open(file))
            y = self.train[1].iloc[id]
            return x, y
        if self.mode=="val":
            image_id = self.val[0].iloc[id]
            file = self.path_images+"/"+image_id+".jpg"
            x = self.preprocess(Image.open(file))
            y = self.val[1].iloc[id]
            return x, y
        if self.mode=="test":
            image_id = self.test[0].iloc[id]
            file = self.path_images+"/"+image_id+".jpg"
            x = self.preprocess(Image.open(file))
            y = self.test[1].iloc[id]
            return x, y
    def __train_val_test_split__(self):
        nval = round(self.frac_val * len(self.image_name))
        ntest = round(self.frac_test * len(self.image_name))
        xtrain, xtest, ytrain, ytest = train_test_split(self.image_name, self.target,
                                                        test_size=ntest, random_state=self.random_state)
        xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain,
                                                      test_size=nval, random_state=self.random_state)
        return xtrain, ytrain, xval, yval, xtest, ytest

The instances of the MelanomaDataset corresponding to the training, validation and evaluation steps are now created and passed to the DataLoader class, which will enable to load the data as batches. Here the batch size is chosen to be 8, mainly for RAM issues (16 was already too much).

In [4]:
path = "../input/melanoma-external-malignant-256"
traindata = MelanomaDataset(path,"train")
valdata = MelanomaDataset(path,"val")
testdata = MelanomaDataset(path,"test")
train = DataLoader(traindata,batch_size=8,shuffle=True)
val = DataLoader(valdata,batch_size=8,shuffle=False)
test = DataLoader(testdata,batch_size=8,shuffle=False)

## Model training

The ResNext50 model is now loaded. Only a tiny modification is performed to its architecture. The original ResNext50 model ends with a linear (or fully connected) layer, which has 2048 input features and outputs 1000 features, since the dataset on which it was trained had 1000 classes. Here, we replace this layer with a layer taking as input 2048 features and outputing 1 feature. The Sigmoid activation function is then added on top, to obtain the probability for each image to correspond to a melanoma.

In [6]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048,out_features=1,bias=True),
    torch.nn.Sigmoid()
)
model.cuda()

The fitting function is now implemented. First, a ModelCheckpoint class is implemented, which will enable to save the trained model at a given epoch if its validation loss is better than the previously saved best model. Its step method will be called after computing the loss on the validation sample, at the end of each epoch. This aims at saving a model as long as it improves itself, and preventing it to overfit (it will eventually overfit, but the best non-overfitted model will be saved).

In [8]:
class ModelCheckpoint():
    def __init__(self, file_save):
        self.val_loss = float("inf")
        self.file_save = file_save
    def step(self, model, val_loss):
        if val_loss < self.val_loss:
            torch.save(model,self.file_save)
            print("Validation loss improved from "+str(round(self.val_loss,2))+
                  " to "+str(round(val_loss,2))+" ; current model saved to "+self.file_save)
            self.val_loss = val_loss

import gc
def fit(model,optim,train,val,n_epochs=10,file_save="/kaggle/working/cnn.pkl"):
    checkpoint = ModelCheckpoint(file_save=file_save)
    loss_fn = torch.nn.BCELoss(reduction="sum")
    for i in range(n_epochs):
        print("Epoch " + str(i + 1) + " - ", end="")
        train_loss = 0
        k = 0
        for input, target in train:
            k = k + 1
            if k % 500 == 0:
                print(str(round(k * 100 / len(train), 2)) + "% - ", end="")
                gc.collect()
            optim.zero_grad()
            output = model(input.cuda())
            loss = loss_fn(output, target.view(-1, 1).to(torch.float32).cuda())
            loss.backward()
            optim.step()
            train_loss = train_loss + loss
        print("Training loss = " + str(round(train_loss.item(), 2)) + " - ", end="")
        with torch.no_grad():
            val_loss = 0
            for input, target in val:
                output = model(input.cuda())
                loss = loss_fn(output, target.view(-1, 1).to(torch.float32).cuda())
                val_loss = val_loss + loss
        print("Validation loss = " + str(round(val_loss.item(), 2)))
        checkpoint.step(model, val_loss.item())
    return model

The fitting process is now launched using the Adam optimizer and 30 epochs.

In [9]:
optim = torch.optim.Adam(model.parameters(),lr=0.001)
model = fit(model,optim,train,val)

The model has finished training itself. The best saved model is reloaded and used to predict the values on the evaluation sample to assess its performance.

In [12]:
model = torch.load("/kaggle/working/cnn.pkl")
pred = []
with torch.no_grad():
    for input, target in test:
        output = model(input.cuda())
        pred.append(output.cpu().numpy())
pred = np.vstack(pred)
print(pd.crosstab(traindata.test[1].values,pred.flatten()>0.5))        

The obtained results are quite satisfying: the true positive rate equals almost 81%, and the false positive rate is only 1.6%. However, there are still almost 20% melanoma which are misclassified. Misclassifying a melanoma as a non-melanoma can be considered to be a worse error than misclassifying a non-melanoma as a melanoma.

The 20% misclassified melanoma can be explained by the important imbalance between classes. Indeed, the non-melanoma are more than 6 times more numerous than the melanoma cases. Thus, the first class is favored over the second one in the training. To counteract this, we can use weights to rebalance the importance in favor of the melanoma class. When computing the loss value, each instance will be weighted depending on its class, and melanoma instances will have a higher contribution to the total loss. To efficiently reduce its loss, the model will have to improve itself on the melanoma class.

The weights are computed using the compute_class_weight function of the Scikit-Learn package. This function gives weights which enable to restore equivalent importance among classes. Everything else in the training process is kept unchanged.

In [13]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048,out_features=1,bias=True),
    torch.nn.Sigmoid()
)
model.cuda()

optim = torch.optim.Adam(model.parameters(),lr=0.001)

from sklearn.utils.class_weight import compute_class_weight
w = compute_class_weight(class_weight="balanced", y=traindata.train[1], classes=[0, 1])

def fit_weighted(model,optim,train,val,weights,n_epochs=10,file_save="/kaggle/working/cnn_weighted.pkl"):
    def make_weight_tensor(ybatch,w0,w1):
        w = np.zeros_like(ybatch)
        w[np.where(ybatch.numpy()==1)[0]] = w1
        w[np.where(ybatch.numpy()==0)[0]] = w0
        return torch.tensor(w)
    checkpoint = ModelCheckpoint(file_save=file_save)
    loss_fn = torch.nn.BCELoss(reduction="none")
    for i in range(n_epochs):
        print("Epoch " + str(i + 1) + " - ", end="")
        train_loss = 0
        k = 0
        for input, target in train:
            k = k + 1
            if k % 500 == 0:
                print(str(round(k * 100 / len(train), 2)) + "% - ", end="")
            optim.zero_grad()
            output = model(input.cuda())
            target = target.view(-1, 1).to(torch.float32)
            loss = loss_fn(output, target.cuda())
            loss = torch.sum(loss*make_weight_tensor(target,w[0],w[1]).cuda())
            loss.backward()
            optim.step()
            train_loss = train_loss + loss
        print("Training loss = " + str(round(train_loss.item(), 2)) + " - ", end="")
        with torch.no_grad():
            val_loss = 0
            yy = []
            pred = []
            for input, target in val:
                output = model(input.cuda())
                target = target.view(-1, 1).to(torch.float32)
                loss = loss_fn(output, target.cuda())
                loss = torch.sum(loss * make_weight_tensor(target, w[0], w[1]).cuda())
                val_loss = val_loss + loss
            print("Validation loss = " + str(round(val_loss.item(), 2)))
        checkpoint.step(model, val_loss.item())
    return model

In [14]:
model = fit_weighted(model,optim,train,val,w)

In [15]:
model = torch.load("/kaggle/working/cnn_weighted.pkl")
pred = []
with torch.no_grad():
    for input, target in test:
        output = model(input.cuda())
        pred.append(output.cpu().numpy())
pred = np.vstack(pred)
print(pd.crosstab(traindata.test[1].values,pred.flatten()>0.5))

A great improvement is obtained here: the model now successfully identifies 89% of the melanoma The number of melanoma correctly identified is improved by almost 10%. On the other hand, the false positive rate has increased to 7%, which was expected.

The algorithm could probably be further improved. For instance, more epochs could be used to train it. Moreover, learning rate scheduling procedures could be used, to adjust the learning rate value during training.